In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# Define the ResNet block
def resnet_block(x, filters, stride):
    y = layers.Conv2D(filters, 3, strides=stride, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)

    y = layers.Conv2D(filters, 3, padding='same')(y)
    y = layers.BatchNormalization()(y)

    if stride != 1:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(x)
    else:
        shortcut = x

    shortcut = layers.BatchNormalization()(shortcut)

    y = layers.Add()([y, shortcut])
    y = layers.Activation('relu')(y)

    return y

# Define the ResNet model
def resnet_model(num_classes):
    inputs = keras.Input(shape=(224, 224, 3))

    x = layers.Conv2D(64, 7, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(2)(x)

    x = resnet_block(x, 64, 2)
    x = resnet_block(x, 128, 2)
    x = resnet_block(x, 256, 2)
    x = resnet_block(x, 512, 2)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, x)
    return model

# Define the data generators
train_dir = 'training'
validation_dir = 'validation'

train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

# Compile the model
model = resnet_model(4)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, steps_per_epoch=train_generator.samples // 32, epochs=10, validation_data=validation_generator, validation_steps=validation_generator.samples // 32)

Found 361 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Epoch 1/10
11/11 [==============================] - 20s 2s/step - loss: 1.4572 - accuracy: 0.5106 - val_loss: 40.7427 - val_accuracy: 0.2083
Epoch 2/10
11/11 [==============================] - 18s 2s/step - loss: 0.8768 - accuracy: 0.6565 - val_loss: 44.9770 - val_accuracy: 0.2292
Epoch 3/10
11/11 [==============================] - 17s 2s/step - loss: 0.7548 - accuracy: 0.7204 - val_loss: 9.1976 - val_accuracy: 0.2500
Epoch 4/10
11/11 [==============================] - 17s 2s/step - loss: 0.7287 - accuracy: 0.7264 - val_loss: 1.1075 - val_accuracy: 0.3854
Epoch 5/10
11/11 [==============================] - 17s 2s/step - loss: 0.6304 - accuracy: 0.7538 - val_loss: 2.3200 - val_accuracy: 0.2812
Epoch 6/10
11/11 [==============================] - 17s 2s/step - loss: 0.6007 - accuracy: 0.7447 - val_loss: 2.0016 - val_accuracy: 0.3646
Epoch 7/10
11/11 [==============================] - 17s 2s/step - loss: 0.46

In [27]:
# Improve
# Define the data generators with data augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Define the ResNet model with dropout and batch normalization
def resnet_model(num_classes):
    inputs = keras.Input(shape=(224, 224, 3))

    x = layers.Conv2D(64, 7, activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)

    x = resnet_block(x, 64, 2)
    x = layers.Dropout(0.2)(x)
    x = resnet_block(x, 128, 2)
    x = layers.Dropout(0.2)(x)
    x = resnet_block(x, 256, 2)
    x = layers.Dropout(0.2)(x)
    x = resnet_block(x, 512, 2)
    x = layers.Dropout(0.2)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, x)
    return model

# Compile the model with a learning rate scheduler
model = resnet_model(4)
model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

lr_scheduler = keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * 0.5 ** (epoch // 5))

# Train the model with early stopping
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
    callbacks=[lr_scheduler, keras.callbacks.EarlyStopping(patience=5)]
)

Epoch 1/10
11/11 [==============================] - 26s 2s/step - loss: 1.3890 - accuracy: 0.5015 - val_loss: 1.2633 - val_accuracy: 0.2708 - lr: 0.0010
Epoch 2/10
11/11 [==============================] - 23s 2s/step - loss: 0.9325 - accuracy: 0.6778 - val_loss: 1.0708 - val_accuracy: 0.5104 - lr: 0.0010
Epoch 3/10
11/11 [==============================] - 22s 2s/step - loss: 0.6949 - accuracy: 0.7477 - val_loss: 1.4215 - val_accuracy: 0.2708 - lr: 0.0010
Epoch 4/10
11/11 [==============================] - 24s 2s/step - loss: 0.6567 - accuracy: 0.7273 - val_loss: 2.0107 - val_accuracy: 0.3021 - lr: 0.0010
Epoch 5/10
11/11 [==============================] - 23s 2s/step - loss: 0.5420 - accuracy: 0.7964 - val_loss: 2.5244 - val_accuracy: 0.1146 - lr: 0.0010
Epoch 6/10
11/11 [==============================] - 22s 2s/step - loss: 0.4469 - accuracy: 0.8116 - val_loss: 1.3438 - val_accuracy: 0.5208 - lr: 5.0000e-04
Epoch 7/10
11/11 [==============================] - 22s 2s/step - loss: 0.3485